<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/drive/1kseAnLcIDHDlO8OyLutzehCQoY41k3Bb#scrollTo=WfzIAI-J6TEO">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/biagiominio/Remote_sensing_2021/blob/main/R_code_classification.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

In [ ]:
url = "https://www.esa.int/var/esa/storage/images/esa_multimedia/images/2020/07/solar_orbiter_s_first_view_of_the_sun2/22133123-1-eng-GB/Solar_Orbiter_s_first_view_of_the_Sun.png"

In [ ]:
library(raster)
library(RStoolbox)

In [ ]:
so <- brick("Solar_Orbiter_s_first_views_of_the_Sun_pillars.jpg")
so

In [ ]:
plotRGB(so, 1,2,3, stretch="lin")


In [ ]:
soc <- unsuperClass(so, nClasses=3)
plot(soc$map)


In [ ]:
soc20 <- unsuperClass(so, nClasses=20)
plot(soc20$map,col=cl)

In [ ]:
cl <- colorRampPalette(c('yellow','black','red'))(100)
plot(soc20$map,col=cl)


Download Solar Orbiter data and proceed further!
Grand Canyon
https://landsat.visibleearth.nasa.gov/view.php?id=80948
When John Wesley Powell led an expedition down the Colorado River and through the Grand Canyon in 1869,

In [ ]:
url = "https://eoimages.gsfc.nasa.gov/images/imagerecords/80000/80948/dolansprings_oli_2013088_canyon_lrg.jpg"

In [ ]:
gc <- brick("dolansprings_oli_2013088_canyon_lrg.jpg")
plotRGB(gc, r=1, g=2, b=3, stretch="lin")


In [ ]:
plotRGB(gc, r=1, g=2, b=3, stretch="hist")

In [ ]:
gcc2 <- unsuperClass(gc, nClasses=2)
gcc2


In [ ]:
plot(gcc2$map)


In [ ]:
gcc4 <- unsuperClass(gc, nClasses=4)
plot(gcc4$map)
